Example from: [https://python.langchain.com/docs/integrations/chat/ollama_functions/](https://python.langchain.com/docs/integrations/chat/ollama_functions/)

In [1]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

model = OllamaFunctions(model="llama3:70b", base_url='http://172.28.105.30/backend', format='json')

In [2]:
class Solution(BaseModel):
    temperature: float = Field(
        ..., description="The temperature of the solution creation")
    atmosphere: str = Field(
        ..., description="The atmosphere of the solution creation")
    method: str = Field(
        ..., description="The method of the solution creation")
    time: float = Field(
        ..., description="The time needed for the solution creation")
    solutes: List[str] = Field(
        ..., description="The solutes used in the solution")
    solute_masses: List[float] = Field(
        ..., description="The masses in miligramm of the solutes used in the solution")
    solvents: List[str] = Field(
        ..., description="The solvents used in the solution")
    solvent_volumes: List[float] = Field(
        ..., description="The volumes in mililiter of the solvents used in the solution")


In [5]:
# Prompt template
prompt = PromptTemplate.from_template(
    """you should extract the information form the following text and create a structured output. If information is missing you should fill it with "None".

Human: {question}
AI: """
)

# Chain
structured_llm = model.with_structured_output(Solution)
chain = prompt | structured_llm


In [6]:
prep = chain.invoke("Please create a solution of 2mg lead iodide as a solute and 100 ml ethanol as a solvent. The solution is produced at 25 degree celsius stirred for 15 minutes.")
prep

Solution(temperature=25.0, atmosphere='None', method='stirred', time=15.0, solutes=['lead iodide'], solute_masses=[2.0], solvents=['ethanol'], solvent_volumes=[100.0])